In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
for n_element in (100,):

    element_score = np.random.normal(size=n_element // 2)

    element_score = pd.Series(
        np.sort(np.concatenate((element_score, -element_score))),
        index=("Element{}".format(i) for i in range(n_element)),
    )

    for n_skip in (0,):

        if n_skip < 1:

            n_skip = int(n_skip * n_element)

        for n_top in (0.1,):

            if n_top < 1:

                n_top = int(n_top * n_element)

            for rate_top_hit in (1,):

                if rate_top_hit < 1:

                    rate_top_hit = int(rate_top_hit * n_top)

                for side in (
                    "-",
                    "+",
                ):

                    set_elements = []

                    if "-" in side:

                        set_elements += element_score.index[
                            n_skip : n_skip + n_top : rate_top_hit
                        ].tolist()

                    if "+" in side:

                        set_elements += element_score.index[
                            -(n_skip + 1) : -(n_skip + 1 + n_top) : -rate_top_hit
                        ][::-1].tolist()

                    for method in ("score pdf k1",):

                        kraft.compute_set_enrichment(
                            element_score,
                            set_elements,
                            method=method,
                            plot_=True,
                            plot=True,
                        )

In [ ]:
element_score = pd.read_csv(
    "benchmark4.gene.list.GSEA.txt",
    sep="\t",
    usecols=("gene.list", "gene.scores"),
    index_col=0,
    squeeze=True,
)

set_elements = kraft.read_gmt("c2.all.v5.1.symbols.gmt")["COLLER_MYC_TARGETS_UP"]

In [ ]:
kraft.compute_set_enrichment(
    element_score, set_elements, method="score pdf k1", plot_=True, plot=True
)